In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 

In [ ]:
from google.colab import files 
files.upload()

Saving Chapter6_Shale Gas Wells.csv to Chapter6_Shale Gas Wells.csv


{'Chapter6_Shale Gas Wells.csv': b'Stage Spacing,bbl/ft,Well Spacing,Dip,Thickness,Lateral Length,Injection Rate,Porosity,ISIP,Water Saturation,Percentage of LG,Pressure Gradient,Proppant Loading,EUR\r\n140,38,850,0,165,8543,64,8.3,8200,17.1,38,0.95,3012,13.3\r\n140,30,900,0,174,8336,60,7.8,8061,16,59.3,0.95,2771,12.5\r\n140,30,900,0,174,9361,65,7.8,8061,16,59.3,0.95,3067,16.9\r\n140,30,950,0,175,9110,69,8.3,7922,15.7,66.9,0.95,3130,16.5\r\n140,30,950,0,175,9310,67,8.3,7922,15.7,66.9,0.95,2991,17.4\r\n140,30,950,0,175,8348,66,8.3,7922,15.7,66.9,0.95,2998,14.9\r\n140,36,950,0,166,7787,64,7.7,7643,17.4,37.1,0.95,2580,13\r\n140,36,950,0,166,8002,56,7.7,7643,17.4,37.1,0.95,2191,14.4\r\n140,36,950,0,166,7276,55,7.7,7643,17.4,37.1,0.94,1566,10.8\r\n140,36,1000,0,166,7777,59,7.7,7643,17.4,37.1,0.94,2309,11.6\r\n140,36,1000,0,166,8277,56,7.7,7643,17.4,37.1,0.95,2115,10.3\r\n140,36,950,0,166,7783,59,7.7,7643,17.4,37.1,0.95,2531,11.6\r\n140,36,950,0,166,7622,71,7.7,7643,17.4,37.1,0.95,2390,12.6\

In [ ]:
df=pd.read_csv('Chapter6_Shale Gas Wells.csv')
df.head()

,Stage Spacing,bbl/ft,Well Spacing,Dip,Thickness,Lateral Length,Injection Rate,Porosity,ISIP,Water Saturation,Percentage of LG,Pressure Gradient,Proppant Loading,EUR
0,140,38,850,0,165,8543,64,8.3,8200,17.1,38.0,0.95,3012,13.3
1,140,30,900,0,174,8336,60,7.8,8061,16.0,59.3,0.95,2771,12.5
2,140,30,900,0,174,9361,65,7.8,8061,16.0,59.3,0.95,3067,16.9
3,140,30,950,0,175,9110,69,8.3,7922,15.7,66.9,0.95,3130,16.5
4,140,30,950,0,175,9310,67,8.3,7922,15.7,66.9,0.95,2991,17.4


In [ ]:
x=df.drop(['EUR'], axis='columns')
y=df['EUR']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.3, random_state=50)

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train=scaler.fit_transform(x_train)
X_test=scaler.transform(x_test)

In [ ]:
!pip install keras-tuner

In [ ]:
from tensorflow import keras
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import  EarlyStopping
from keras_tuner import Hyperband , RandomSearch


In [ ]:
def model(hp):
  model=Sequential()
  for i in range(hp.Int('number_of_layers', 2,10)):
    hp_units=hp.Int('units'+str(i), min_value=2 ,max_value=9, step=1)
    model.add(Dense(units=hp_units, activation='relu'))
  
  model.add(Dense(1))
  hp_loss=hp.Choice('loss', values=['mse', 'msle'])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1,1e-2,1e-3])), loss=hp_loss)
  return model







In [ ]:
tuner=RandomSearch(hypermodel=model,objective='val_loss', seed=100, max_trials=5)
early_stop=EarlyStopping(patience=2)
tuner.search(X_train, y_train, callbacks=[early_stop], epochs=100, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_loss: 11.401200294494629

Best val_loss So Far: 0.02051529847085476
Total elapsed time: 00h 00m 32s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hp=tuner.get_best_hyperparameters()[0]



In [ ]:
h_model=tuner.hypermodel.build(best_hp)

#h_model.summary()

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
number_of_layers: 3
units0: 6
units1: 3
loss: msle
learning_rate: 0.1
units2: 2
Score: 0.02051529847085476
Trial summary
Hyperparameters:
number_of_layers: 2
units0: 9
units1: 8
loss: mse
learning_rate: 0.1
units2: 9
units3: 6
units4: 6
units5: 4
units6: 5
units7: 5
units8: 5
Score: 9.008628845214844
Trial summary
Hyperparameters:
number_of_layers: 7
units0: 6
units1: 3
loss: mse
learning_rate: 0.1
units2: 7
units3: 6
units4: 6
units5: 8
units6: 5
units7: 8
units8: 3
Score: 11.401200294494629
Trial summary
Hyperparameters:
number_of_layers: 9
units0: 5
units1: 2
loss: mse
learning_rate: 0.01
units2: 8
units3: 4
units4: 7
units5: 8
units6: 6
units7: 3
units8: 6
Score: 26.166662216186523
Trial summary
Hyperparameters:
number_of_layers: 9
units0: 7
units1: 4
loss: mse
learning_rate: 0.01
units2: 9
units3: 2
units4: 2
units5: 2
units6: 2
units7: 2
units8: 2
Score: 26.191572189331055


In [ ]:
#####
tuner=Hyperband(model, objective='loss', seed=100)
early_stop=EarlyStopping(patience=2)
tuner.search(X_train, y_train, callbacks=[early_stop], epochs=10, validation_data=(X_test, y_test))

Trial 246 Complete [00h 00m 02s]
loss: 8.603270530700684

Best loss So Far: 0.01141613069921732
Total elapsed time: 00h 00m 55s

Search: Running Trial #247

Value             |Best Value So Far |Hyperparameter
10                |3                 |number_of_layers
8                 |8                 |units0
2                 |8                 |units1
msle              |msle              |loss
0.001             |0.1               |learning_rate
6                 |7                 |units2
4                 |7                 |units3
4                 |2                 |units4
8                 |2                 |units5
6                 |8                 |units6
6                 |7                 |units7
4                 |3                 |units8
8                 |4                 |units9
100               |34                |tuner/epochs
34                |12                |tuner/initial_epoch
1                 |4                 |tuner/bracket
1                 |3         

KeyboardInterrupt: ignored

In [ ]:
best_hp=tuner.get_best_hyperparameters()[0]

In [ ]:
h_model = tuner.hypermodel.build(best_hp)
h_model.summary()

ValueError: ignored

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
number_of_layers: 3
units0: 8
units1: 8
loss: msle
learning_rate: 0.1
units2: 7
units3: 7
units4: 2
units5: 2
units6: 8
units7: 7
units8: 3
units9: 4
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 0136
Score: 0.01141613069921732
Trial summary
Hyperparameters:
number_of_layers: 7
units0: 6
units1: 7
loss: msle
learning_rate: 0.01
units2: 4
units3: 7
units4: 6
units5: 4
units6: 8
units7: 3
units8: 6
units9: 8
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 3
tuner/round: 2
tuner/trial_id: 0192
Score: 0.013460501097142696
Trial summary
Hyperparameters:
number_of_layers: 3
units0: 8
units1: 8
loss: msle
learning_rate: 0.1
units2: 7
units3: 7
units4: 2
units5: 2
units6: 8
units7: 7
units8: 3
units9: 4
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0145
Score: 0.014118047431111336
Trial summary
Hyperpa

In [ ]:
h_model.fit(X_train, y_train)
h_model.evaluate(X_test,y_test)

5/5 [==============================] - 0s 5ms/step - loss: 0.0324


0.03244728967547417

In [ ]:
yp=h_model.predict(X_test)

In [ ]:
x_test

,Stage Spacing,bbl/ft,Well Spacing,Dip,Thickness,Lateral Length,Injection Rate,Porosity,ISIP,Water Saturation,Percentage of LG,Pressure Gradient,Proppant Loading
365,150,30,700,0,140,4500,58,6.6,5000,24.1,79.7,0.93,2888
313,141,30,800,0,164,8128,59,7.5,7783,17.2,64.4,0.95,2842
461,148,30,750,0,141,8276,58,6.6,5000,24.1,79.7,0.95,2451
158,143,30,700,0,156,7860,55,6.4,7643,19.1,32.9,0.93,2928
333,140,30,1000,0,168,8195,71,8.0,7643,15.7,79.7,0.95,2971
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,140,73,900,0,181,10257,72,8.3,7783,15.7,32.9,0.95,3079
423,155,30,700,0,154,7909,59,6.6,5000,24.1,79.7,0.75,1951
9,140,36,1000,0,166,7777,59,7.7,7643,17.4,37.1,0.94,2309
18,142,30,800,0,165,7042,71,7.7,7783,17.3,86.5,0.90,2623


In [ ]:
X_test

array([[0.05263158, 0.        , 0.07142857, ..., 0.80875   , 0.9       ,
        0.84728341],
       [0.00526316, 0.        , 0.21428571, ..., 0.6175    , 1.        ,
        0.8247675 ],
       [0.04210526, 0.        , 0.14285714, ..., 0.80875   , 1.        ,
        0.63338228],
       ...,
       [0.        , 0.13333333, 0.5       , ..., 0.27625   , 0.95      ,
        0.56387665],
       [0.01052632, 0.        , 0.21428571, ..., 0.89375   , 0.75      ,
        0.7175722 ],
       [0.03684211, 0.        , 0.07142857, ..., 0.80875   , 0.9       ,
        0.89916789]])

In [ ]:
df.iloc[461]

Stage Spacing         148.00
bbl/ft                 30.00
Well Spacing          750.00
Dip                     0.00
Thickness             141.00
Lateral Length       8276.00
Injection Rate         58.00
Porosity                6.60
ISIP                 5000.00
Water Saturation       24.10
Percentage of LG       79.70
Pressure Gradient       0.95
Proppant Loading     2451.00
EUR                    11.20
Name: 461, dtype: float64

In [ ]:
yp

array([[11.385195 ],
       [13.028933 ],
       [12.607843 ],
       [11.54164  ],
       [14.984877 ],
       [14.129645 ],
       [18.463379 ],
       [11.966316 ],
       [11.550547 ],
       [15.537987 ],
       [12.417159 ],
       [15.058711 ],
       [11.648198 ],
       [17.82492  ],
       [12.532294 ],
       [10.998987 ],
       [12.332449 ],
       [15.832169 ],
       [11.115616 ],
       [14.329707 ],
       [13.190278 ],
       [14.71292  ],
       [13.398107 ],
       [11.821033 ],
       [16.161104 ],
       [11.999166 ],
       [10.800102 ],
       [15.523528 ],
       [13.802885 ],
       [15.025719 ],
       [10.1976385],
       [10.767542 ],
       [14.785567 ],
       [15.553713 ],
       [14.957958 ],
       [14.681603 ],
       [12.334968 ],
       [12.993986 ],
       [13.56903  ],
       [14.365095 ],
       [17.0244   ],
       [11.998114 ],
       [10.645039 ],
       [14.415147 ],
       [10.211215 ],
       [11.716128 ],
       [14.21265  ],
       [17.51